In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *

sys.path.insert(0, "../../scripts/methods/")
from my_moscot import moscot_align, moscot_align_ref
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

_XSERVTransSocketUNIXCreateListener: ...SocketCreateListener() failed
_XSERVTransMakeAllCOTSServerListeners: server already running
(EE) 
Fatal server error:
(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) 


In [2]:
order = [
    155, 153,151,149,147,144,142,140, 138, 143,141,139, 137,135,133,130,131,
129,127,125,123,121,119,117,115,113,111,109,107,105,103, 101, 99, 97, 95, 93,
91,89,87,85,81,83,79,77,75,73,71,67,65,63,61,59,57,55,53,51,49,47,45,43,41,39,
37,33,31,29,27,25,28,30,32,34,36,38,40,42,44,48,50,52,54,56,58,60,62,
    64,66,68,72,74,76,78,80,82,84,88,90,92,94,96,98,100,102,104,106,108,
    110,112,114,116,118,120,122,124,136,134,132,128,126
]
data_folder = "/dm7100f/yifan/StereoSeq_macaque_processed"
files = [f'macaque_T{o}.h5ad' for o in order]

In [3]:
# data_folder = "./results/split_data/"
results_folder = "./results/Moscot/"
figures_folder = "./results/figures/Moscot"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(figures_folder).mkdir(parents=True, exist_ok=True)

In [4]:
## Perform the Spateo alignment
from tqdm import tqdm
import time
sampling_num = 20000
spatial_key = 'r_spatial'
key_added = 'align_spatial'
for i in tqdm(range(len(files)-1)):
    slice1 = ad.read_h5ad(os.path.join(data_folder, files[i]))
    slice2 = ad.read_h5ad(os.path.join(data_folder, files[i+1]))
    slice1.obsm[spatial_key] = slice1.obsm[spatial_key].astype(np.float64)
    slice2.obsm[spatial_key] = slice2.obsm[spatial_key].astype(np.float64)
    slice1 = slice1[~np.isnan(slice1.obsm[spatial_key].sum(1))]
    slice2 = slice2[~np.isnan(slice2.obsm[spatial_key].sum(1))]
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    top_genes_num = 5000
    top_genes_1 = np.argsort(-slice1.X.sum(0)).A[0][:top_genes_num] if slice1.shape[1] > top_genes_num else np.arange(slice1.shape[1])
    top_genes_2 = np.argsort(-slice2.X.sum(0)).A[0][:top_genes_num] if slice2.shape[1] > top_genes_num else np.arange(slice2.shape[1])
    slice1 = slice1[:,top_genes_1]
    slice2 = slice2[:,top_genes_2]
    
    time_start = time.time()
    align_slices, pis = moscot_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="cuda",
        verbose=False,
        alpha=0.5,
    )
    # align_slices[1].uns['align_spatial_iter'] = morpho_model.iter_added
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                      | 0/1 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(20000, 20000)].                                        


2024-07-11 23:54:52.702717: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
100%|██████████████████████████████| 1/1 [00:34<00:00, 34.72s/it]
